# conda 25.5.1
# Python 3.10.18
# torch 2.2.2（pytorch）
# pip install transformers==4.38.2
# pip install accelerate==0.30.0
# pip install peft==0.17.0

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

# 我们选择一个在中文语料上预训练过的GPT-2模型
model_name = "uer/gpt2-chinese-cluecorpussmall" 

# AutoTokenizer 会自动加载速度更快的 FastTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=8,  # 低秩维度，越小越轻量
    lora_alpha=32,
    target_modules=["c_attn", "c_proj", "mlp.c_fc", "mlp.c_proj"],  # GPT-2 的注意力层
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# 使用 PEFT 库将 LoRA 配置应用到模型上
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 103,248,384 || trainable%: 1.1425341049405675


/Users/lichao/miniconda3/envs/LLMs/lib/python3.10/site-packages/peft/tuners/lora/model.py:347: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [12]:
#查看你目前机子所支持的GPU类型
import torch
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("检测到MPS可用，将使用MPS进行训练。")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("检测到CUDA可用，将使用GPU进行训练。")
else:
    device = torch.device("cpu")
    print("未检测到MPS，将使用CPU进行训练。")


检测到MPS可用，将使用MPS进行训练。


In [ ]:
from datasets import load_dataset

# 定义每个文本块的大小
BLOCK_SIZE = 128

# 训练数据集文件路径
TRAIN_FILE="cleaned_huagaiji.txt"

# 加载数据集
# 假设你的数据集是一个文本文件，每行是一个样本
raw_datasets = load_dataset('text', data_files={'train': TRAIN_FILE})

# 定义一个函数来对文本进行分词
def tokenize_function(samples):
    return tokenizer(samples["text"]) 

# 对数据集进行分词
# 使用 batched=True 以批处理的方式进行分词，这样可以提高效率
# remove_columns=["text"] 用于移除原始文本列，只保留分词的结果
tokenized_datasets = raw_datasets.map(
    tokenize_function, batched=True, remove_columns=["text"]
)

# 定义一个函数来将文本分块
# 这里的 BLOCK_SIZE 是每个块的大小
# 例如，如果 BLOCK_SIZE=128，那么每个块将包含 128 个 token
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()} 
    #examples{
    # 'input_ids':[[101, 5678, ...], [102, 1314...],...],
    # 'attention_mask':[[1,1,...], [1,1...]...],
    # ...
    #}
    #concatenated_examples {
    # 'input_ids':[101, 5678, ...,102, 1314...]
    # 'attention_mask':[1,1....1,1...]
    #}
    total_length = len(concatenated_examples[list(examples.keys())[0]]) 
    total_length = (total_length // BLOCK_SIZE) * BLOCK_SIZE 
    result = {
        k: [t[i : i + BLOCK_SIZE] for i in range(0, total_length, BLOCK_SIZE)] 
        for k, t in concatenated_examples.items() 
    }
    # result {
    #   'input_ids':[[...BLOCK_SIZE个ID...], [...BLOCK_SIZE个ID...] ...],
    #   'attention_mask':[[...BLOCK_SIZE个1...], [...BLOCK_SIZE个1...] ...],
    #   ...
    #}
    result["labels"] = result["input_ids"].copy() 
    # result {
    #   'input_ids':[[...BLOCK_SIZE个ID...], [...BLOCK_SIZE个ID...] ...],
    #   'attention_mask':[[...BLOCK_SIZE个1...], [...BLOCK_SIZE个1...] ...],
    #   ...
    #   'labels':[[...BLOCK_SIZE个ID...], [...BLOCK_SIZE个ID...] ...]
    #}
    return result

# 对分词后的数据集进行分块
# 使用 batched=True 以批处理的方式进行分块，这样可以提高效率
# batch_size=1000 用于指定每次处理的样本数量
lm_datasets = tokenized_datasets.map(
    group_texts, batched=True, batch_size=1000
)

print(lm_datasets)



DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 461
    })
})


In [14]:
from transformers import TrainingArguments

OUTPUT_DIR = "./gpt2-luxun-finetuned-mps"
NUM_TRAIN_EPOCHS = 3
PER_DEVICE_TRAIN_BATCH_SIZE = 4
LEARNING_RATE = 5e-5
SAVE_STEPS = 500
OVERWRITE_OUTPUT_DIR = True

# 设置训练参数,这些参数可以根据你的需求进行调整
# 例如，num_train_epochs 可以设置为你希望的训练轮数
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=OVERWRITE_OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    save_steps=SAVE_STEPS,
    learning_rate=1e-5, #LEARNING_RATE,
    logging_steps=50,  # 每50步记录一次日志
    evaluation_strategy="steps",  # 每500步评估一次
    eval_steps=500,  # 每500步评估一次
    save_total_limit=2,  # 最多保存2个checkpoint
    warmup_ratio=0.03,  # 预热比例
    use_mps_device=True  
)

In [15]:
from transformers import Trainer,DataCollatorForLanguageModeling

# 使用 DataCollatorForLanguageModeling 来处理数据
# 在构建每个 batch 时自动进行动态填充（padding）、掩码生成（masking）、以及标签对齐。
# 这里的 mlm=False 表示我们不使用掩码语言模型（Masked Language Modeling），因为我们是在做自回归语言模型（Causal Language Modeling）
# 如果你使用的是 BERT 或其他需要掩码的模型，可以将 mlm 设置为 True
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 创建 Trainer 实例
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

print(f"--- 开始训练 ---")
trainer.train()

/Users/lichao/miniconda3/envs/LLMs/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


--- 开始训练 ---


  0%|          | 0/348 [00:00<?, ?it/s]

{'loss': 4.456, 'grad_norm': 1.3484209775924683, 'learning_rate': 8.842729970326411e-06, 'epoch': 0.43}
{'loss': 4.3656, 'grad_norm': 1.470889925956726, 'learning_rate': 7.359050445103858e-06, 'epoch': 0.86}
{'loss': 4.2514, 'grad_norm': 1.7284166812896729, 'learning_rate': 5.875370919881306e-06, 'epoch': 1.29}
{'loss': 4.146, 'grad_norm': 1.8330525159835815, 'learning_rate': 4.391691394658754e-06, 'epoch': 1.72}
{'loss': 4.1199, 'grad_norm': 1.7699863910675049, 'learning_rate': 2.9080118694362018e-06, 'epoch': 2.16}
{'loss': 4.0692, 'grad_norm': 1.455802321434021, 'learning_rate': 1.42433234421365e-06, 'epoch': 2.59}
{'train_runtime': 181.6624, 'train_samples_per_second': 7.613, 'train_steps_per_second': 1.916, 'train_loss': 4.20465280817843, 'epoch': 3.0}


TrainOutput(global_step=348, training_loss=4.20465280817843, metrics={'train_runtime': 181.6624, 'train_samples_per_second': 7.613, 'train_steps_per_second': 1.916, 'train_loss': 4.20465280817843, 'epoch': 3.0})

In [16]:
# 8. 保存最终模型 (与之前完全相同)
print(f"--- 训练完成，保存最终模型到 {OUTPUT_DIR} ---")
trainer.save_model()
tokenizer.save_pretrained(OUTPUT_DIR)

print("--- 脚本执行完毕 ---")

--- 训练完成，保存最终模型到 ./gpt2-luxun-finetuned-mps ---
--- 脚本执行完毕 ---


In [17]:

MODEL_PATH = "./gpt2-luxun-finetuned-mps" 

# 自动检测并设置设备 (对于您的Mac，这将优先选择MPS)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"正在使用设备: MPS (Apple Silicon GPU)")
else:
    device = torch.device("cpu")
    print(f"正在使用设备: CPU")

print("正在加载分词器...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

print("正在加载训练好的模型...")
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)

model.to(device)
print("模型加载并移动到设备完成！")

正在使用设备: MPS (Apple Silicon GPU)
正在加载分词器...
正在加载训练好的模型...
模型加载并移动到设备完成！


In [18]:
def generate_text(prompt_text, max_len=150):
    print(f"\n--- 正在为以下prompt生成文本 ---\n'{prompt_text}'")
    
    # 编码：将您的起始句子转换为模型可理解的ID，并移动到设备上
    inputs = tokenizer(prompt_text, return_tensors="pt").to(device)

    # 生成：调用模型的 .generate() 方法
    outputs = model.generate(
        **inputs,
        max_length=200,
        num_return_sequences=1,  # 生成几个不同的结果
        do_sample=True,          # 开启采样，让文本更具创造性
        top_k=50,                # 从概率最高的50个词中选择
        top_p=0.95,              # 从概率总和为95%的词汇中选择
        temperature=0.8,         # 控制生成的随机性，值越小越保守
        repetition_penalty=1.5
    )

    # 解码：将生成的ID转换回人类可读的文本
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print("\n--- 生成结果 ---")
    print(generated_text)

prompt1 = "秋天的后半夜，月亮下去了，太阳还没有出，只剩下一片乌蓝的天空。"
generate_text(prompt1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- 正在为以下prompt生成文本 ---
'秋天的后半夜，月亮下去了，太阳还没有出，只剩下一片乌蓝的天空。'

--- 生成结果 ---
秋 天 的 后 半 夜 ， 月 亮 下 去 了 ， 太 阳 还 没 有 出 ， 只 剩 下 一 片 乌 蓝 的 天 空 。 书 上 写 着 ： 从 前 看 过 ， 今 日 之 故 人 也 ！ 那 是 我 们 这 个 世 界 最 大 、 最 古 老 和 最 残 酷 的 朝 代 时 期 但 它 却 在 今 年 来 临 ， 终 于 要 离 开 了.. 又 何 妨 呢 ？ - - - - 《 中 国 文 史 》 里 面 的 语 句 ， 不 知 道 你 听 懂 多 少 ？ 我 先 问 清 楚 ， 如 果 说 得 通 透 点 儿 就 好 了 ！ 这 样 子 算 么 ？ 还 真 难 讲 了 ， 作 为 历 史 家 ， 咱 要 想 办 法 让 读 者 明 白 该 怎 么 回 答 或 者 应 用 哪 种 方 式 ， 而 且 还 必 须 记 住 这 两 点 ：


In [10]:
# list=[1,2,3,4, 5]
# print(*list)

dict = {
    "key1":[1,1,1,1,1],
    "key2":[2,2,2,2,2]
}

def process_data(key1, key2):
    print(key1)
    print(key2)

process_data(**dict)

[1, 1, 1, 1, 1]
[2, 2, 2, 2, 2]
